<a href="https://colab.research.google.com/github/Qhwelah/NeuralBlock-NeuralNetPredictor/blob/main/NeuralNetPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip3 uninstall gdown -y && pip3 install gdown
import gdown

url = "https://drive.google.com/uc?id=1UPXnC2oy20r4Yesw7lPuDRHlsoR0l0DV"
output = "data.csv"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1UPXnC2oy20r4Yesw7lPuDRHlsoR0l0DV
To: /content/data.csv
100%|██████████| 8.96M/8.96M [00:00<00:00, 48.7MB/s]


'data.csv'

In [2]:
import pandas as pd

df = pd.read_csv("data.csv")

print(df)

      xRot   yRot                block_1  block_1_dist                block_2  \
0    81.45  26.98   minecraft:oak_leaves             6   minecraft:oak_planks   
1    59.47   2.02   minecraft:oak_planks             4        minecraft:water   
2   -58.81   3.46         minecraft:dirt           159        minecraft:stone   
3    61.45  36.54   minecraft:oak_leaves            40        minecraft:stone   
4   -44.43  84.42  minecraft:grass_block            56        minecraft:stone   
..     ...    ...                    ...           ...                    ...   
115  23.38  11.33   minecraft:oak_planks            11        minecraft:water   
116   1.92  50.17         minecraft:dirt            72   minecraft:oak_planks   
117 -64.33  83.93   minecraft:oak_planks            21  minecraft:grass_block   
118  -1.77   5.55        minecraft:stone           146        minecraft:stone   
119  11.65  76.97          minecraft:air           113   minecraft:oak_planks   

     block_2_dist          

In [3]:
print(df["block_1_dist"][2])

159


In [9]:
## Getting all values of non-numeric types only
df.info()
non_numeric_values = df.select_dtypes(exclude=["float", "int"])
# print(non_numeric_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Columns: 7202 entries, xRot to block_3600_dist
dtypes: float64(2), int64(7200)
memory usage: 6.6 MB


In [5]:
## Mapping all non-numeric data values to numeric values
block_mapping = df.block_1.factorize()
print("Conversion values: ")
print(block_mapping)
print()

block_mapping_conversions = []

print("Event log for block conversion pushes: ")
i = 0
while i < len(block_mapping[0]):
    included = False
    if block_mapping[0][i] < len(block_mapping_conversions):
      included = True
    if not included:
      new_conversion = block_mapping[1][block_mapping[0][i]]
      block_mapping_conversions.append(new_conversion)
      print(f"pushing {new_conversion} to values at index {len(block_mapping_conversions)-1}")
    i += 1

block_mapping_conversions.append('unknown')

print()
print("Final conversion values: ")
# print(block_mapping_two)
print(block_mapping_conversions)

Conversion values: 
(array([0, 1, 2, 0, 3, 4, 5, 6, 5, 1, 1, 5, 5, 5, 7, 4, 4, 5, 1, 5, 1, 5,
       5, 2, 2, 5, 5, 4, 7, 3, 6, 4, 1, 1, 1, 2, 7, 5, 0, 5, 1, 6, 7, 5,
       1, 4, 1, 5, 5, 7, 1, 5, 1, 4, 1, 6, 5, 2, 4, 5, 6, 5, 7, 5, 1, 5,
       5, 5, 4, 4, 5, 5, 5, 4, 5, 4, 4, 6, 5, 4, 1, 1, 6, 1, 1, 5, 5, 5,
       4, 2, 1, 1, 4, 1, 4, 1, 5, 5, 4, 5, 5, 1, 1, 0, 5, 5, 5, 1, 5, 4,
       2, 6, 1, 5, 2, 1, 2, 1, 5, 4]), Index(['minecraft:oak_leaves', 'minecraft:oak_planks', 'minecraft:dirt',
       'minecraft:grass_block', 'minecraft:air', 'minecraft:stone',
       'minecraft:water', 'minecraft:oak_log'],
      dtype='object'))

Event log for block conversion pushes: 
pushing minecraft:oak_leaves to values at index 0
pushing minecraft:oak_planks to values at index 1
pushing minecraft:dirt to values at index 2
pushing minecraft:grass_block to values at index 3
pushing minecraft:air to values at index 4
pushing minecraft:stone to values at index 5
pushing minecraft:water to values at in

In [8]:
## Converting actual data values to numbers using block mapping
def convertBlock(val):
  for i in range(len(block_mapping_conversions)):
    if val == block_mapping_conversions[i]:
      return i
  return -1

block_columns = [col for col in df.columns if col.startswith("block_") and not col.endswith("_dist")]
print(block_columns)
df_original = df.copy() # make a backup copy of the dataframe to reference later
for col in block_columns:
  df[col] = df[col].apply(convertBlock)

print()
print(df) # the numerized dataframe

['block_1', 'block_2', 'block_3', 'block_4', 'block_5', 'block_6', 'block_7', 'block_8', 'block_9', 'block_10', 'block_11', 'block_12', 'block_13', 'block_14', 'block_15', 'block_16', 'block_17', 'block_18', 'block_19', 'block_20', 'block_21', 'block_22', 'block_23', 'block_24', 'block_25', 'block_26', 'block_27', 'block_28', 'block_29', 'block_30', 'block_31', 'block_32', 'block_33', 'block_34', 'block_35', 'block_36', 'block_37', 'block_38', 'block_39', 'block_40', 'block_41', 'block_42', 'block_43', 'block_44', 'block_45', 'block_46', 'block_47', 'block_48', 'block_49', 'block_50', 'block_51', 'block_52', 'block_53', 'block_54', 'block_55', 'block_56', 'block_57', 'block_58', 'block_59', 'block_60', 'block_61', 'block_62', 'block_63', 'block_64', 'block_65', 'block_66', 'block_67', 'block_68', 'block_69', 'block_70', 'block_71', 'block_72', 'block_73', 'block_74', 'block_75', 'block_76', 'block_77', 'block_78', 'block_79', 'block_80', 'block_81', 'block_82', 'block_83', 'block_84', 